In [1]:
suppressPackageStartupMessages({
  library(dplyr)
  library(dataorganizer)
  library(devtools)
  library(cellAdmix)
  library(ggplot2)
  library(readr)
  devtools::load_all()
})

suppressMessages({
  load_arial_font()
})

theme_set(theme_bw())

ℹ Loading cellAdmixNotebooks


In [3]:
base_dir <- CachePath('nsclc_scaled_dat11/')

# load data
cell_annot <- prepare_nsclc_metadata(reps='all')
df <- prepare_nsclc_transcript_data(cell_annot,reps='all')

# save a counts version of the data
so_spatial_orig <- get_counts_meta_seurat(df, normalize=TRUE)

Normalizing layer: counts



In [4]:
write_rds(so_spatial_orig, file.path(base_dir,'so_spatial_orig.rds'))

In [5]:
k <- 15
nmf_res <- readRDS(file=paste0(base_dir,'nmf_joint_k',k,'.rds'))

In [6]:
# loading precomputed bridge-results to determine factor-ctypes to remove
fp_checks <- readRDS(file=paste0(base_dir,'fp_checks.rds'))
annot_res_all <- fp_checks[[1]]
f_rm <- annot_res_all[[paste0('joint_',k,'_bridge')]]

In [9]:
### first generating cleaned fibroblasts
df_ct <- df[df$celltype=='fibroblast',]

crf_res <- run_crf_all(
  df_ct, nmf_res, num.nn=10, same.label.ratio=5,
  normalize.by='gene',n.cores=20
)

# clean the df
df_ct$factor <- crf_res[,1]

df_cln <- df_ct[!(paste(df_ct$factor, df_ct$celltype, sep = "_") %in% f_rm), ]

# make a counts version of df_cln
so_spatial_cln <- get_counts_meta_seurat(df_cln,normalize=TRUE)

Normalizing layer: counts



In [10]:
write_rds(so_spatial_cln, paste0(base_dir,'so_spatial_cln_',k,'_fib_full.rds'))

In [11]:
### first generating cleaned macrophages
df_ct <- df[df$celltype=='macrophage',]

crf_res <- run_crf_all(
  df_ct, nmf_res, num.nn=10, same.label.ratio=5,
  normalize.by='gene', n.cores=20
)

# clean the df
df_ct$factor <- crf_res[,1]

df_cln <- df_ct[!(paste(df_ct$factor, df_ct$celltype, sep = "_") %in% f_rm), ]

# make a counts version of df_cln
so_spatial_cln <- get_counts_meta_seurat(df_cln,normalize=TRUE)

Normalizing layer: counts



In [12]:
write_rds(so_spatial_cln,file=paste0(base_dir,'so_spatial_cln_',k,'_macro_full.rds'))